<a href="https://colab.research.google.com/github/abhinav70291/TechnicalAnalysis-for-intra-day-trading/blob/main/ML_Part_of_Technical_Analysis_correlation_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q --upgrade ipywidgets matplotlib
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, SelectMultiple, SelectionRangeSlider
from datetime import datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.0 MB/s eta 0:00:00


In [ ]:

from scipy.signal import argrelextrema

In [ ]:
df=pd.read_csv("/content/Manappuram_10minute.csv")
df.tail(5)

,Date,Open,High,Low,Close,Volume
2492,2020-04-28T14:35:00,123.90,124.25,123.7,124.15,289726
2493,2020-04-28T14:45:00,124.15,125.80,124.1,125.80,1706683
2494,2020-04-28T14:55:00,125.80,125.80,125.8,125.80,145240
2495,2020-04-28T15:05:00,125.80,131.50,125.8,130.85,3162404
2496,2020-04-28T15:15:00,130.95,130.95,129.6,129.75,221556


In [ ]:
# Converting Date column into datetime dftype
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)


# **State the Values you want to consider for this backtesting and technical analysis**

In [ ]:
# Initialize a DataFrame to store metrics for each parameter
metrics_df = pd.DataFrame(columns=['indicator', 'hit_rate', 'profit_factor', 'sharpe_ratio', 'max_drawdown', 'total_profit', 'profit_percentage', 'avg_win_to_avg_loss_ratio', 'sortino_ratio', 'num_trades'])

# **Defining ALL indicator functions  here**

In [ ]:
# calculates SMA using "n" as the rolling window size
def ma_calc(df, n):
  df["sma"] = df.Close.rolling(window=n).mean()
  return df

# calculates SMA using "m" and "n" as the rolling window size
def longshort_ma_calc(df, m, n):
  df["sma_short"] = df.Close.rolling(window=min(m,n)).mean()
  df["sma_long"] = df.Close.rolling(window=max(m,n)).mean()
  return df

# calculates EMA using "n" as the rolling window size
def ema_calc(df, n):
  df["ema"] = df.Close.ewm(span=n, adjust=False).mean()
  return df


def longshort_ema_calc(df, m, n):
  df["ema_short"] = df.Close.ewm(span=min(m,n), adjust=False).mean()
  df["ema_long"] = df.Close.ewm(span=max(m,n), adjust=False).mean()
  return df

# calculates RSI using "n" as the lookback period
def rsi_calc(df, n):
  df['rsi'] = 100 - (100 / (1 + df['Close'].diff().apply(lambda x: x if x > 0 else 0).rolling(window=n).mean() / df['Close'].diff().apply(lambda x: -x if x < 0 else 0).rolling(window=n).mean()))
  return df

# calculates OBV
def obv_calc(df):
  df['obv'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
  return df

def bb_calc(df, n):
  df["sma"] = df.Close.rolling(window=n).mean()
  df["std"] = df.Close.rolling(window=n).std()
  df["upper_bb"] = df["sma"] + (2 * df["std"])
  df["lower_bb"] = df["sma"] - (2 * df["std"])
  return df

# Volume weighted average price
def vwap_calc(df):
    df['vwap'] = (df['Volume'] * df['Close']).cumsum() / df['Volume'].cumsum()
    return df

# Supertrend Indicator
def supertrend_calc(df, period, multiplier):
    # Calculate basic upper and lower bands
    df['hl_avg'] = (df['High'] + df['Low']) / 2
    df['range'] = df['High'] - df['Low']
    df['upper_band'] = df['hl_avg'] + multiplier * df['range']
    df['lower_band'] = df['hl_avg'] - multiplier * df['range']

    # Calculate final upper and lower bands
    df['upper_band_final'] = np.where((df['upper_band'] < df['upper_band'].shift(1)) | (df['Close'] > df['upper_band'].shift(1)), df['upper_band'], df['upper_band'].shift(1))
    df['lower_band_final'] = np.where((df['lower_band'] > df['lower_band'].shift(1)) | (df['Close'] < df['lower_band'].shift(1)), df['lower_band'], df['lower_band'].shift(1))

    # Calculate Supertrend
    df['supertrend'] = np.where(df['Close'] <= df['upper_band_final'], df['upper_band_final'], df['lower_band_final'])
    df['supertrend'] = np.where(df['Close'] >= df['lower_band_final'], df['lower_band_final'], df['supertrend'])

    return df

# calculates Average Directional Index (ADX)
def adx_calc(df, n):
    df['hl_diff'] = df['High'] - df['Low']
    df['hc_diff'] = abs(df['High'] - df['Close'].shift(1))
    df['lc_diff'] = abs(df['Low'] - df['Close'].shift(1))
    df['tr'] = df[['hl_diff', 'hc_diff', 'lc_diff']].max(axis=1)
    df['+dm'] = np.where((df['High'] > df['High'].shift(1)) & (df['High'] - df['High'].shift(1) > df['Low'].shift(1) - df['Low']), df['High'] - df['High'].shift(1), 0)
    df['-dm'] = np.where((df['Low'] < df['Low'].shift(1)) & (df['High'].shift(1) - df['High'] < df['Low'].shift(1) - df['Low']), df['Low'].shift(1) - df['Low'], 0)
    df['tr_ema'] = df['tr'].ewm(span=n, adjust=False).mean()
    df['+dm_ema'] = df['+dm'].ewm(span=n, adjust=False).mean()
    df['-dm_ema'] = df['-dm'].ewm(span=n, adjust=False).mean()
    df['+di'] = (df['+dm_ema'] / df['tr_ema']) * 100
    df['-di'] = (df['-dm_ema'] / df['tr_ema']) * 100
    df['dx'] = (abs(df['+di'] - df['-di']) / (df['+di'] + df['-di'])) * 100
    df['adx'] = df['dx'].rolling(window=n).mean()

    return df

# calculates MACD
def macd_calc(df, short_n, long_n, signal_n):
    df['ema_short'] = df['Close'].ewm(span=short_n, adjust=False).mean()
    df['ema_long'] = df['Close'].ewm(span=long_n, adjust=False).mean()
    df['macd_line'] = df['ema_short'] - df['ema_long']
    df['signal_line'] = df['macd_line'].ewm(span=signal_n, adjust=False).mean()
    df['macd_histogram'] = df['macd_line'] - df['signal_line']

    return df

def find_extrema(df, n):
    df['min'] = df.iloc[argrelextrema(df['Close'].values, np.less_equal, order=n)[0]]['Close']
    df['max'] = df.iloc[argrelextrema(df['Close'].values, np.greater_equal, order=n)[0]]['Close']
    return df


# **Data Preparation**

In [ ]:
df=find_extrema(df,16)
df=ma_calc(df,524)
df=longshort_ma_calc(df,564,4)
df=ema_calc(df,63)
df=longshort_ema_calc(df,374,5)
df=rsi_calc(df,118)
df=adx_calc(df,32)

# columns_to_drop = ['hl_diff', 'hc_diff', 'lc_diff', 'tr', '+dm', '-dm', 'tr_ema', '+dm_ema', '-dm_ema', '+di', '-di', 'dx']

# # Drop the columns
# df = df.drop(columns=columns_to_drop)


In [ ]:
 # Initialize the signal column with hold signals
df['true_signal'] = 0

# Generate buy signals at local minima
df.loc[df['min'].notna(), 'true_signal'] = 1

# Generate sell signals at local maxima
df.loc[df['max'].notna(), 'true_signal'] = -1


In [ ]:
df.true_signal.value_counts()

 0    2390
 1      55
-1      52
Name: true_signal, dtype: int64

In [ ]:
# As the buy/sell signals themselves are sparse,I'd predict positions instead of signals
for day in np.unique(df.index.date):
    indices = df[df.index.date == day].index
    for i in range(len(indices) - 1):  # subtracting 1 because we're looking ahead by 1 row
        if df.loc[indices[i], "true_signal"] == 1 and df.loc[indices[i + 1], "true_signal"] == 0:
            df.loc[indices[i + 1], "true_signal"] = 1
        elif df.loc[indices[i], "true_signal"] == -1 and df.loc[indices[i + 1], "true_signal"] == 0:
            df.loc[indices[i + 1], "true_signal"] = -1


In [ ]:
df.true_signal.value_counts()

 0    971
-1    811
 1    715
Name: true_signal, dtype: int64

the last position of the data, if at all, one is in position , should be to "Sell". but using "Sell" position as the last transaction data will be of no use in prediction as the features(indicators) used for prediction do not contain information about the trade being the last trade of the day. "Sell" position can be inforced manually also, as a postprocessing, after the ML Model makes it's predictions

In [ ]:
df=df.fillna(-1) # basically replacing all Nan values iwth -1 since ML/DL models do not work with Null data
df

,Open,High,Low,Close,Volume,min,max,sma,sma_short,sma_long,...,+dm,-dm,tr_ema,+dm_ema,-dm_ema,+di,-di,dx,adx,true_signal
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-21 09:15:00,179.30,180.20,178.25,180.15,173897,-1.0,-1.00,-1.000000,-1.0000,-1.000000,...,0.00,0.0,1.950000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,0
2020-01-21 09:25:00,180.00,181.30,180.00,180.50,175277,-1.0,-1.00,-1.000000,-1.0000,-1.000000,...,1.10,0.0,1.910606,0.066667,0.000000,3.489294,0.000000,100.000000,-1.000000,0
2020-01-21 09:35:00,180.50,181.05,180.25,180.55,110920,-1.0,-1.00,-1.000000,-1.0000,-1.000000,...,0.00,0.0,1.843297,0.062626,0.000000,3.397514,0.000000,100.000000,-1.000000,0
2020-01-21 09:45:00,180.55,181.50,180.05,181.35,80456,-1.0,181.35,-1.000000,180.6375,-1.000000,...,0.45,0.2,1.819460,0.086103,0.012121,4.732362,0.666198,75.319414,-1.000000,-1
2020-01-21 09:55:00,181.35,181.65,181.00,181.30,73996,-1.0,-1.00,-1.000000,180.9250,-1.000000,...,0.15,0.0,1.748584,0.089976,0.011387,5.145648,0.651189,77.532943,-1.000000,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-28 14:35:00,123.90,124.25,123.70,124.15,289726,-1.0,-1.00,107.540935,123.8500,106.497872,...,0.00,0.0,0.773377,0.248342,0.026264,32.111333,3.396051,80.871298,79.639203,0
2020-04-28 14:45:00,124.15,125.80,124.10,125.80,1706683,-1.0,-1.00,107.598187,124.3500,106.556915,...,1.55,0.0,0.829536,0.327230,0.024673,39.447359,2.974254,85.977650,80.222201,0
2020-04-28 14:55:00,125.80,125.80,125.80,125.80,145240,-1.0,-1.00,107.656489,124.8875,106.616046,...,0.00,0.0,0.779261,0.307398,0.023177,39.447359,2.974254,85.977650,80.587223,0


# **TRAIN-TEST-SPLIT**
A validation set of 10 days and a train set of 55 days should do for this dataset, let's see how it goes

In [ ]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'min', 'max', 'sma',
       'sma_short', 'sma_long', 'ema', 'ema_short', 'ema_long', 'rsi',
       'hl_diff', 'hc_diff', 'lc_diff', 'tr', '+dm', '-dm', 'tr_ema',
       '+dm_ema', '-dm_ema', '+di', '-di', 'dx', 'adx', 'true_signal'],
      dtype='object')

In [ ]:
df["longshort_sma_delta"]=df["sma_long"]-df["sma_short"]
df["longshort_ema_delta"]=df["ema_long"]-df["ema_short"]
df["Close_sma_delta"]=df["Close"]-df["sma_short"]
df["Close_ema_delta"]=df["Close"]-df["ema_short"]

In [ ]:
train=df[:38*55]
test=df[38*56:]

In [ ]:
len(test)

369

In [ ]:
target=train.true_signal
indicators=train.drop(["true_signal"],axis=1)

In [ ]:
val_target=test.true_signal
val_indicators=test.drop(["true_signal"],axis=1)

In [ ]:
target.value_counts()

 0    869
-1    651
 1    570
Name: true_signal, dtype: int64

In [ ]:
indicators.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'min', 'max', 'sma',
       'sma_short', 'sma_long', 'ema', 'ema_short', 'ema_long', 'rsi',
       'hl_diff', 'hc_diff', 'lc_diff', 'tr', '+dm', '-dm', 'tr_ema',
       '+dm_ema', '-dm_ema', '+di', '-di', 'dx', 'adx', 'longshort_sma_delta',
       'longshort_ema_delta', 'Close_sma_delta', 'Close_ema_delta'],
      dtype='object')

In [ ]:
# # !pip -q install flaml
# from flaml import AutoML
# automl = AutoML()
# automl.fit(indicators, target, X_val=val_indicators,y_val=val_target,task="classification", max_iter=500,metric='micro_f1',estimator_list=["lgbm"])


Some more feature engineering is required here:
I'd like to introduce these indicators also-

1)Body Size

2)Upper Shadow

3)Lower Shadow

4) Candle Direction

5) Price Range

6)Relative Price/ normalized price : $$\frac{{\text{{close}} - \text{{low}}}}{{\text{{high}} - \text{{low}}}}$$



In [ ]:
# 1) Body Size
df['body_size'] = abs(df['Close'] - df['Open'])

# 2) Upper Shadow
df['upper_shadow'] = df['High'] - df[['Open', 'Close']].max(axis=1)

# 3) Lower Shadow
df['lower_shadow'] = df[['Open', 'Close']].min(axis=1) - df['Low']

# 4) Candle Direction
df['candle_direction'] = np.where(df['Close'] >= df['Open'], '1', '-1')

# 5) Price Range
df['price_range'] = df['High'] - df['Low']

# 6) Relative Price / Normalized Price
df['relative_price'] = (df['Close'] - df['Low']) / (df['High'] - df['Low'])


In [ ]:
df=df.drop(["Close","Open","High","Low"],axis=1)

In [ ]:
train=df[:38*55]
test=df[38*56:]

In [ ]:
len(test)

369

In [ ]:
target=train.true_signal
indicators=train.drop(["true_signal"],axis=1)

In [ ]:
val_target=test.true_signal
val_indicators=test.drop(["true_signal"],axis=1)

In [ ]:
target.value_counts()

 0    869
-1    651
 1    570
Name: true_signal, dtype: int64

In [ ]:
!pip -q install flaml
from flaml import AutoML
automl = AutoML()
automl.fit(indicators, target, X_val=val_indicators,y_val=val_target,task="classification", max_iter=500,metric='micro_f1',estimator_list=["lgbm"])


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 6.1 MB/s eta 0:00:00
[flaml.automl.logger: 03-18 08:18:15] {1680} INFO - task = classification
[flaml.automl.logger: 03-18 08:18:15] {1688} INFO - Data split method: stratified
[flaml.automl.logger: 03-18 08:18:15] {1691} INFO - Evaluation method: holdout
[flaml.automl.logger: 03-18 08:18:15] {1789} INFO - Minimizing error metric: 1-micro_f1
[flaml.automl.logger: 03-18 08:18:15] {1901} INFO - List of ML learners in AutoML Run: ['lgbm']
[flaml.automl.logger: 03-18 08:18:15] {2219} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 03-18 08:18:15] {2345} INFO - Estimated sufficient time budget=10000s. Estimated necessary time budget=10s.
[flaml.automl.logger: 03-18 08:18:15] {2392} INFO -  at 0.4s,	estimator lgbm's best error=0.5962,	best estimator lgbm's best error=0.5962
[flaml.automl.logger: 03-18 08:18:15] {2219} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 03-18 08:18:15] {2392} INFO -  at 0.5

no particular increase in f1 score , not a great improvement

Remember, each of the iterations is being evaluated on the test set, if it were evaluated on the train set, (which i've tried) , it gives very less error (0.07 ), that means that there is severe overfitting going on here . One thing that we can do, is to use same indicator with multiple values, sma_5, sma_10, sma_15 etc and similar for other indicators

In [ ]:
df=pd.read_csv("/content/Manappuram_10minute.csv")
df.tail(5)

,Date,Open,High,Low,Close,Volume
2492,2020-04-28T14:35:00,123.90,124.25,123.7,124.15,289726
2493,2020-04-28T14:45:00,124.15,125.80,124.1,125.80,1706683
2494,2020-04-28T14:55:00,125.80,125.80,125.8,125.80,145240
2495,2020-04-28T15:05:00,125.80,131.50,125.8,130.85,3162404
2496,2020-04-28T15:15:00,130.95,130.95,129.6,129.75,221556


In [ ]:
# Converting Date column into datetime dftype
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)


In [ ]:
def calculate_indicators(df):
    base = 19
    for i in range(5, 31, 5):  # This will loop over 5, 10, 15
        n = base * i

        # Calculate SMA
        df[f'sma_{i}'] = df['Close'].rolling(window=n).mean()

        # Calculate EMA
        df[f'ema_{i}'] = df['Close'].ewm(span=n, adjust=False).mean()

        # Calculate RSI
        delta = df['Close'].diff()
        up, down = delta.copy(), delta.copy()
        up[up < 0] = 0
        down[down > 0] = 0
        avg_gain = up.rolling(window=n).mean()
        avg_loss = abs(down.rolling(window=n).mean())
        rs = avg_gain / avg_loss
        df[f'rsi_{i}'] = 100 - (100 / (1 + rs))

    # Calculate long-short SMA and EMA deltas
    for i in range(10, 16, 5):  # This will loop over 10, 15
        df[f'sma_{i}_5_delta'] = df[f'sma_{i}'] - df['sma_5']
        df[f'ema_{i}_5_delta'] = df[f'ema_{i}'] - df['ema_5']

    return df

def calculate_adx(df):
    base = 19
    for i in range(5, 31, 5):  # This will loop over 5, 10, 15
        n = base * i

        # Calculate the components of ADX
        df['hl_diff'] = df['High'] - df['Low']
        df['hc_diff'] = abs(df['High'] - df['Close'].shift(1))
        df['lc_diff'] = abs(df['Low'] - df['Close'].shift(1))
        df['tr'] = df[['hl_diff', 'hc_diff', 'lc_diff']].max(axis=1)
        df['+dm'] = np.where((df['High'] > df['High'].shift(1)) & (df['High'] - df['High'].shift(1) > df['Low'].shift(1) - df['Low']), df['High'] - df['High'].shift(1), 0)
        df['-dm'] = np.where((df['Low'] < df['Low'].shift(1)) & (df['High'].shift(1) - df['High'] < df['Low'].shift(1) - df['Low']), df['Low'].shift(1) - df['Low'], 0)
        df['tr_ema'] = df['tr'].ewm(span=n, adjust=False).mean()
        df['+dm_ema'] = df['+dm'].ewm(span=n, adjust=False).mean()
        df['-dm_ema'] = df['-dm'].ewm(span=n, adjust=False).mean()
        df['+di'] = (df['+dm_ema'] / df['tr_ema']) * 100
        df['-di'] = (df['-dm_ema'] / df['tr_ema']) * 100
        df['dx'] = (abs(df['+di'] - df['-di']) / (df['+di'] + df['-di'])) * 100

        # Calculate ADX
        df[f'adx_{i}'] = df['dx'].rolling(window=n).mean()

    # Drop the intermediate columns
    df = df.drop(columns=['hl_diff', 'hc_diff', 'lc_diff', 'tr', '+dm', '-dm', 'tr_ema', '+dm_ema', '-dm_ema', '+di', '-di', 'dx'])

    return df

def calculate_bollinger_bands(df):
    base = 19
    for i in range(5, 31, 5):  # This will loop over 5, 10, 15
        n = base * i

        # Calculate Simple Moving Average (SMA)
        df[f'sma_{i}'] = df['Close'].rolling(window=n).mean()

        # Calculate Standard Deviation
        df[f'std_{i}'] = df['Close'].rolling(window=n).std()

        # Calculate Upper Bollinger Band = SMA + 2*std
        df[f'upper_bb_{i}'] = df[f'sma_{i}'] + 2*df[f'std_{i}']

        # Calculate Lower Bollinger Band = SMA - 2*std
        df[f'lower_bb_{i}'] = df[f'sma_{i}'] - 2*df[f'std_{i}']

    return df

# Candlestick indicators

# 1) Body Size
df['body_size'] = abs(df['Close'] - df['Open'])

# 2) Upper Shadow
df['upper_shadow'] = df['High'] - df[['Open', 'Close']].max(axis=1)

# 3) Lower Shadow
df['lower_shadow'] = df[['Open', 'Close']].min(axis=1) - df['Low']

# 4) Candle Direction
df['candle_direction'] = np.where(df['Close'] >= df['Open'], '1', '-1')

# 5) Price Range
df['price_range'] = df['High'] - df['Low']

# 6) Relative Price / Normalized Price
df['relative_price'] = (df['Close'] - df['Low']) / (df['High'] - df['Low'])



In [ ]:
df=calculate_bollinger_bands(df)
df=calculate_adx(df)
df=calculate_indicators(df)

In [ ]:
# List of necessary indicators
indicators = ['Open', 'High', 'Low', 'Volume', 'body_size', 'upper_shadow',
              'lower_shadow', 'candle_direction', 'price_range', 'relative_price',
              'sma_5', 'std_5', 'upper_bb_5', 'lower_bb_5', 'sma_10', 'std_10',
              'upper_bb_10', 'lower_bb_10', 'sma_15', 'std_15', 'upper_bb_15',
              'lower_bb_15', 'sma_20', 'std_20', 'upper_bb_20', 'lower_bb_20',
              'sma_25', 'std_25', 'upper_bb_25', 'lower_bb_25', 'sma_30', 'std_30',
              'upper_bb_30', 'lower_bb_30', 'adx_5', 'adx_10', 'adx_15', 'adx_20',
              'adx_25', 'adx_30', 'ema_5', 'rsi_5', 'ema_10', 'rsi_10', 'ema_15',
              'rsi_15', 'ema_20', 'rsi_20', 'ema_25', 'rsi_25', 'ema_30', 'rsi_30']

# Calculate delta for each indicator
for indicator in indicators:
    try:
        df[indicator + '_delta'] = df['Close'] - df[indicator]
    except:
        pass


In [ ]:
df.head(5)

,Open,High,Low,Close,Volume,body_size,upper_shadow,lower_shadow,candle_direction,price_range,...,ema_10_delta,rsi_10_delta,ema_15_delta,rsi_15_delta,ema_20_delta,rsi_20_delta,ema_25_delta,rsi_25_delta,ema_30_delta,rsi_30_delta
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-21 09:15:00,179.30,180.20,178.25,180.15,173897,0.85,0.05,1.05,1,1.95,...,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN,0.000000,NaN
2020-01-21 09:25:00,180.00,181.30,180.00,180.50,175277,0.50,0.80,0.00,1,1.30,...,0.346335,NaN,0.347552,NaN,0.348163,NaN,0.348529,NaN,0.348774,NaN
2020-01-21 09:35:00,180.50,181.05,180.25,180.55,110920,0.05,0.50,0.25,1,0.80,...,0.392185,NaN,0.394772,NaN,0.396073,NaN,0.396855,NaN,0.397377,NaN
2020-01-21 09:45:00,180.55,181.50,180.05,181.35,80456,0.80,0.15,0.50,1,1.45,...,1.179701,NaN,1.186417,NaN,1.189794,NaN,1.191826,NaN,1.193183,NaN
2020-01-21 09:55:00,181.35,181.65,181.00,181.30,73996,0.05,0.30,0.30,-1,0.65,...,1.117872,NaN,1.128470,NaN,1.133811,NaN,1.137029,NaN,1.139179,NaN


In [ ]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'body_size', 'upper_shadow',
       'lower_shadow', 'candle_direction', 'price_range',
       ...
       'ema_10_delta', 'rsi_10_delta', 'ema_15_delta', 'rsi_15_delta',
       'ema_20_delta', 'rsi_20_delta', 'ema_25_delta', 'rsi_25_delta',
       'ema_30_delta', 'rsi_30_delta'],
      dtype='object', length=108)

Calculating delta columns

In [ ]:
# Calculate the difference between short and long term moving averages
df['sma_delta_10_5'] = df['sma_10'] - df['sma_5']
df['sma_delta_15_5'] = df['sma_15'] - df['sma_5']
df['sma_delta_15_10'] = df['sma_15'] - df['sma_10']

df['ema_delta_10_5'] = df['ema_10'] - df['ema_5']
df['ema_delta_15_5'] = df['ema_15'] - df['ema_5']
df['ema_delta_15_10'] = df['ema_15'] - df['ema_10']

# Calculate the difference between the close price and the Bollinger Bands
df['bb_upper_delta_5'] = df['Close'] - df['upper_bb_5']
df['bb_lower_delta_5'] = df['Close'] - df['lower_bb_5']
df['bb_upper_delta_10'] = df['Close'] - df['upper_bb_10']
df['bb_lower_delta_10'] = df['Close'] - df['lower_bb_10']
df['bb_upper_delta_15'] = df['Close'] - df['upper_bb_15']
df['bb_lower_delta_15'] = df['Close'] - df['lower_bb_15']

# Calculate the difference between consecutive RSI values
df['rsi_delta_5'] = df['rsi_5'].diff()
df['rsi_delta_10'] = df['rsi_10'].diff()
df['rsi_delta_15'] = df['rsi_15'].diff()

df=find_extrema(df,16)


In [ ]:
 # Initialize the signal column with hold signals
df['true_signal'] = 0

# Generate buy signals at local minima
df.loc[df['min'].notna(), 'true_signal'] = 1

# Generate sell signals at local maxima
df.loc[df['max'].notna(), 'true_signal'] = -1


In [ ]:
# As the buy/sell signals themselves are sparse,I'd predict positions instead of signals
for day in np.unique(df.index.date):
    indices = df[df.index.date == day].index
    for i in range(len(indices) - 1):  # subtracting 1 because we're looking ahead by 1 row
        if df.loc[indices[i], "true_signal"] == 1 and df.loc[indices[i + 1], "true_signal"] == 0:
            df.loc[indices[i + 1], "true_signal"] = 1
        elif df.loc[indices[i], "true_signal"] == -1 and df.loc[indices[i + 1], "true_signal"] == 0:
            df.loc[indices[i + 1], "true_signal"] = -1


In [ ]:
df.true_signal.value_counts()

 0    971
-1    811
 1    715
Name: true_signal, dtype: int64

In [ ]:
df=df.drop(["Close","Open","High","Low"],axis=1)

In [ ]:
train=df[:38*55]
test=df[38*56:]

In [ ]:
len(test)

369

In [ ]:
target=train.true_signal
indicators=train.drop(["true_signal"],axis=1)

In [ ]:
val_target=test.true_signal
val_indicators=test.drop(["true_signal"],axis=1)

In [ ]:
target.value_counts()

 0    869
-1    651
 1    570
Name: true_signal, dtype: int64

In [ ]:
indicators

,Volume,body_size,upper_shadow,lower_shadow,candle_direction,price_range,relative_price,sma_5,std_5,upper_bb_5,...,bb_lower_delta_5,bb_upper_delta_10,bb_lower_delta_10,bb_upper_delta_15,bb_lower_delta_15,rsi_delta_5,rsi_delta_10,rsi_delta_15,min,max
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-21 09:15:00,173897,0.85,0.05,1.05,1,1.95,0.974359,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-21 09:25:00,175277,0.50,0.80,0.00,1,1.30,0.384615,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-21 09:35:00,110920,0.05,0.50,0.25,1,0.80,0.375000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-21 09:45:00,80456,0.80,0.15,0.50,1,1.45,0.896552,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,181.35
2020-01-21 09:55:00,73996,0.05,0.30,0.30,-1,0.65,0.461538,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-13 10:05:00,297543,0.15,1.20,0.15,1,1.50,0.200000,107.907368,4.237020,116.381408,...,5.866672,-11.213542,19.979858,-8.564646,19.191663,0.304692,-0.188859,1.069314,NaN,NaN
2020-04-13 10:15:00,129428,0.05,0.45,0.30,1,0.80,0.437500,107.962105,4.168778,116.299661,...,5.725450,-11.208960,19.974223,-8.505183,19.239218,-0.199710,-0.382472,-0.213734,NaN,NaN
2020-04-13 10:25:00,121791,0.25,0.20,0.25,-1,0.70,0.357143,108.006316,4.115010,116.236337,...,5.323705,-11.499984,19.662090,-8.720288,18.977832,-0.596168,0.090801,-0.678898,NaN,NaN


In [ ]:
indicators.describe()

,Volume,body_size,upper_shadow,lower_shadow,price_range,relative_price,sma_5,std_5,upper_bb_5,lower_bb_5,...,bb_lower_delta_5,bb_upper_delta_10,bb_lower_delta_10,bb_upper_delta_15,bb_lower_delta_15,rsi_delta_5,rsi_delta_10,rsi_delta_15,min,max
count,2.090000e+03,2090.000000,2090.000000,2090.000000,2090.000000,2089.000000,1996.000000,1996.000000,1996.000000,1996.000000,...,1996.000000,1901.000000,1901.000000,1806.000000,1806.000000,1994.000000,1899.000000,1804.000000,46.000000,42.000000
mean,2.366107e+05,0.604019,0.308804,0.304976,1.217799,0.488059,148.057497,3.733102,155.523701,140.591294,...,5.802619,-15.782745,7.829085,-22.462478,9.249027,-0.000967,0.000625,-0.002538,146.477174,153.477381
std,3.646028e+05,0.735147,0.356938,0.393170,0.997896,0.299187,35.354572,2.423437,32.809489,38.345968,...,6.429551,14.916858,8.691123,19.404115,9.973438,1.184498,0.596721,0.391738,37.198318,34.322177
min,5.013000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,85.459474,0.906248,92.425246,72.818410,...,-10.804056,-71.600212,-12.152515,-84.704183,-15.476870,-7.545010,-4.493211,-3.322090,78.000000,85.500000
25%,8.407100e+04,0.150000,0.100000,0.100000,0.650000,0.230769,104.907500,1.895916,116.829920,93.503399,...,1.605020,-22.123631,1.595159,-29.508247,1.715990,-0.539917,-0.288483,-0.189216,105.612500,121.450000
50%,1.556850e+05,0.400000,0.200000,0.200000,0.950000,0.475000,164.634211,2.999542,168.858211,160.435665,...,3.907697,-13.655573,5.829042,-18.689312,6.885418,0.000000,0.000000,-0.009924,163.375000,167.725000
75%,2.819530e+05,0.750000,0.400000,0.400000,1.450000,0.750000,176.477895,5.102265,180.572294,170.171927,...,8.428868,-4.446294,12.116043,-7.063831,16.203196,0.603220,0.279681,0.190045,176.762500,179.512500
max,1.239266e+07,9.500000,4.500000,6.650000,13.450000,1.000000,189.210526,13.632513,193.441800,186.225425,...,36.231998,9.607164,41.602836,6.559190,42.916599,7.762635,4.793982,3.010510,187.300000,192.150000


In [ ]:
# try:
#     indicators=np.log(indicators)
#     val_indicators=np.log(val_indicators)
# except:
#     pass

In [ ]:
# !pip -q install flaml
from flaml import AutoML
automl = AutoML()
automl.fit(indicators, target, X_val=val_indicators,y_val=val_target, max_iter=1000,metric='micro_f1',estimator_list=["lgbm"])

[flaml.automl.logger: 03-18 08:19:15] {1680} INFO - task = classification
[flaml.automl.logger: 03-18 08:19:15] {1688} INFO - Data split method: stratified
[flaml.automl.logger: 03-18 08:19:15] {1691} INFO - Evaluation method: holdout
[flaml.automl.logger: 03-18 08:19:15] {1789} INFO - Minimizing error metric: 1-micro_f1
[flaml.automl.logger: 03-18 08:19:15] {1901} INFO - List of ML learners in AutoML Run: ['lgbm']
[flaml.automl.logger: 03-18 08:19:15] {2219} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 03-18 08:19:15] {2345} INFO - Estimated sufficient time budget=10000s. Estimated necessary time budget=10s.
[flaml.automl.logger: 03-18 08:19:15] {2392} INFO -  at 0.9s,	estimator lgbm's best error=0.6287,	best estimator lgbm's best error=0.6287
[flaml.automl.logger: 03-18 08:19:15] {2219} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 03-18 08:19:16] {2392} INFO -  at 1.0s,	estimator lgbm's best error=0.6287,	best estimator lgbm's best error=0.6287
[fl

This didn't work any better obviously. Same results. the next method would be to decompose the closing price into 4 signals using wavelet transform, and use each of those 4 signals to generate indicators using window sizes from our analysis.

Moreover, I will also be using these features for each of the closing prices, which will tell us about the moving naturee of each of the 4 resultant closing prices.

**Mean**: The average value of the closing prices in the window.

**Harmonic Mean**: The reciprocal of the arithmetic mean of the reciprocals of the closing prices in the window.

**Standard Deviation**: Measures the amount of variation or dispersion of the closing prices in the window.

**Variance**: The square of the standard deviation.
Kurtosis: Describes the shape of the probability distribution of the closing prices in the window.


**Root Mean Square (RMS)**: The square root of the mean square (the arithmetic mean of the squares of the closing prices).

**Shape Factor:** The ratio of the RMS to the mean.

**Peak Value:** The maximum closing price in the window.

**Peak-to-Peak Value:** The difference between the maximum and minimum closing prices in the window.

**Interquartile Range**: The range between the first quartile (25th percentile) and the third quartile (75th percentile) of the closing prices in the window.

**Shannon Entropy:** A measure of the uncertainty or randomness of the closing prices in the window.
Summation: The total sum of the closing prices in the window.

**Max Frequency, Peak Frequency, Bandwidth:** These are frequency-domain features that can be calculated from the Fast Fourier Transform (FFT) of the closing prices.

**Signal-to-Noise Ratio (SNR):** The ratio of the power of a signal (meaningful information) and the power of background noise (unwanted signal).

**Spectral Entropy:** Entropy of the power spectral density (PSD) of the closing prices.

In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy.stats import skew, kurtosis
import pywt

In [ ]:
df=pd.read_csv("/content/Manappuram_10minute.csv")
df.tail(5)
# Converting Date column into datetime dftype
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)


In [ ]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

In [ ]:
import numpy as np
import scipy.stats as stats
import scipy.signal as signal
from scipy.fft import fft

# Define the window size
window_size = 38*3

# Calculate the mean
df['Mean'] = df['Close'].rolling(window=window_size).mean()

# Calculate the harmonic mean
df['Harmonic_Mean'] = df['Close'].rolling(window=window_size).apply(lambda x: stats.hmean(x))

# Calculate the standard deviation
df['Standard_Deviation'] = df['Close'].rolling(window=window_size).std()

# Calculate the variance
df['Variance'] = df['Close'].rolling(window=window_size).var()


df['Slope'] = df['Close'].rolling(window=window_size).apply(lambda x: (x.max() - x.min()) / window_size)
df['Slope_Start_End'] = df['Close'].rolling(window=window_size).apply(lambda x: (x[-1] - x[0]) / (window_size - 1))



# Calculate the kurtosis
df['Kurtosis'] = df['Close'].rolling(window=window_size).apply(lambda x: stats.kurtosis(x))

# Calculate the root mean square (RMS)
df['RMS'] = df['Close'].rolling(window=window_size).apply(lambda x: np.sqrt(np.mean(np.square(x))))

# Calculate the shape factor
df['Shape_Factor'] = df['RMS'] / df['Mean']

# Calculate the peak value
df['Peak_Value'] = df['Close'].rolling(window=window_size).max()

# Calculate the peak-to-peak value
df['Peak_to_Peak_Value'] = df['Close'].rolling(window=window_size).apply(lambda x: np.ptp(x))

# Calculate the interquartile range
df['Interquartile_Range'] = df['Close'].rolling(window=window_size).apply(lambda x: np.subtract(*np.percentile(x, [75, 25])))

# Calculate the Shannon entropy
df['Shannon_Entropy'] = df['Close'].rolling(window=window_size).apply(lambda x: stats.entropy(x))

# Calculate the summation
df['Summation'] = df['Close'].rolling(window=window_size).sum()

# Calculate the signal-to-noise ratio (SNR)
df['SNR'] = df['Close'].rolling(window=window_size).apply(lambda x: np.mean(x) / np.std(x))

# Calculate the spectral entropy
# df['Spectral_Entropy'] = df['Close'].rolling(window=window_size).apply(lambda x: stats.entropy(np.abs(fft(x))))


In [ ]:
import numpy as np
import scipy.stats as stats
import scipy.signal as signal
from scipy.fft import fft

# Define the window size
window_size = 38*3

# Calculate the mean
df['Volume_Mean'] = df['Volume'].rolling(window=window_size).mean()

# Calculate the harmonic mean
df['Volume_Harmonic_Mean'] = df['Volume'].rolling(window=window_size).apply(lambda x: stats.hmean(x))

# Calculate the standard deviation
df['Volume_Standard_Deviation'] = df['Volume'].rolling(window=window_size).std()

# Calculate the variance
df['Volume_Variance'] = df['Volume'].rolling(window=window_size).var()


df['Volume_Slope'] = df['Volume'].rolling(window=window_size).apply(lambda x: (x.max() - x.min()) / window_size)
df['Volume_Slope_Start_End'] = df['Volume'].rolling(window=window_size).apply(lambda x: (x[-1] - x[0]) / (window_size - 1))



# Calculate the kurtosis
df['Volume_Kurtosis'] = df['Volume'].rolling(window=window_size).apply(lambda x: stats.kurtosis(x))

# Calculate the root mean square (RMS)
df['Volume_RMS'] = df['Volume'].rolling(window=window_size).apply(lambda x: np.sqrt(np.mean(np.square(x))))

# Calculate the shape factor
df['Volume_Shape_Factor'] = df['RMS'] / df['Mean']

# Calculate the peak value
df['Volume_Peak_Value'] = df['Volume'].rolling(window=window_size).max()

# Calculate the peak-to-peak value
df['Volume_Peak_to_Peak_Value'] = df['Volume'].rolling(window=window_size).apply(lambda x: np.ptp(x))

# Calculate the interquartile range
df['Volume_Interquartile_Range'] = df['Volume'].rolling(window=window_size).apply(lambda x: np.subtract(*np.percentile(x, [75, 25])))

# Calculate the Shannon entropy
df['Volume_Shannon_Entropy'] = df['Volume'].rolling(window=window_size).apply(lambda x: stats.entropy(x))

# Calculate the summation
df['Volume_Summation'] = df['Volume'].rolling(window=window_size).sum()

# Calculate the signal-to-noise ratio (SNR)
df['Volume_SNR'] = df['Volume'].rolling(window=window_size).apply(lambda x: np.mean(x) / np.std(x))

# Calculate the spectral entropy
# df['Spectral_Entropy'] = df['Volume'].rolling(window=window_size).apply(lambda x: stats.entropy(np.abs(fft(x))))



In [ ]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Mean', 'Harmonic_Mean',
       'Standard_Deviation', 'Variance', 'Slope', 'Slope_Start_End',
       'Kurtosis', 'RMS', 'Shape_Factor', 'Peak_Value', 'Peak_to_Peak_Value',
       'Interquartile_Range', 'Shannon_Entropy', 'Summation', 'SNR',
       'Volume_Mean', 'Volume_Harmonic_Mean', 'Volume_Standard_Deviation',
       'Volume_Variance', 'Volume_Slope', 'Volume_Slope_Start_End',
       'Volume_Kurtosis', 'Volume_RMS', 'Volume_Shape_Factor',
       'Volume_Peak_Value', 'Volume_Peak_to_Peak_Value',
       'Volume_Interquartile_Range', 'Volume_Shannon_Entropy',
       'Volume_Summation', 'Volume_SNR'],
      dtype='object')

In [ ]:
# calculates SMA using "n" as the rolling window size
def ma_calc(df, n):
  df["sma"] = df.Close.rolling(window=n).mean()
  return df

# calculates SMA using "m" and "n" as the rolling window size
def longshort_ma_calc(df, m, n):
  df["sma_short"] = df.Close.rolling(window=min(m,n)).mean()
  df["sma_long"] = df.Close.rolling(window=max(m,n)).mean()
  return df

# calculates EMA using "n" as the rolling window size
def ema_calc(df, n):
  df["ema"] = df.Close.ewm(span=n, adjust=False).mean()
  return df


def longshort_ema_calc(df, m, n):
  df["ema_short"] = df.Close.ewm(span=min(m,n), adjust=False).mean()
  df["ema_long"] = df.Close.ewm(span=max(m,n), adjust=False).mean()
  return df

# calculates RSI using "n" as the lookback period
def rsi_calc(df, n):
  df['rsi'] = 100 - (100 / (1 + df['Close'].diff().apply(lambda x: x if x > 0 else 0).rolling(window=n).mean() / df['Close'].diff().apply(lambda x: -x if x < 0 else 0).rolling(window=n).mean()))
  return df

# calculates OBV
def obv_calc(df):
  df['obv'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
  return df

def bb_calc(df, n):
  df["sma"] = df.Close.rolling(window=n).mean()
  df["std"] = df.Close.rolling(window=n).std()
  df["upper_bb"] = df["sma"] + (2 * df["std"])
  df["lower_bb"] = df["sma"] - (2 * df["std"])
  return df

# Volume weighted average price
def vwap_calc(df):
    df['vwap'] = (df['Volume'] * df['Close']).cumsum() / df['Volume'].cumsum()
    return df

# Supertrend Indicator
def supertrend_calc(df, period, multiplier):
    # Calculate basic upper and lower bands
    df['hl_avg'] = (df['High'] + df['Low']) / 2
    df['range'] = df['High'] - df['Low']
    df['upper_band'] = df['hl_avg'] + multiplier * df['range']
    df['lower_band'] = df['hl_avg'] - multiplier * df['range']

    # Calculate final upper and lower bands
    df['upper_band_final'] = np.where((df['upper_band'] < df['upper_band'].shift(1)) | (df['Close'] > df['upper_band'].shift(1)), df['upper_band'], df['upper_band'].shift(1))
    df['lower_band_final'] = np.where((df['lower_band'] > df['lower_band'].shift(1)) | (df['Close'] < df['lower_band'].shift(1)), df['lower_band'], df['lower_band'].shift(1))

    # Calculate Supertrend
    df['supertrend'] = np.where(df['Close'] <= df['upper_band_final'], df['upper_band_final'], df['lower_band_final'])
    df['supertrend'] = np.where(df['Close'] >= df['lower_band_final'], df['lower_band_final'], df['supertrend'])

    return df

# calculates Average Directional Index (ADX)
def adx_calc(df, n):
    df['hl_diff'] = df['High'] - df['Low']
    df['hc_diff'] = abs(df['High'] - df['Close'].shift(1))
    df['lc_diff'] = abs(df['Low'] - df['Close'].shift(1))
    df['tr'] = df[['hl_diff', 'hc_diff', 'lc_diff']].max(axis=1)
    df['+dm'] = np.where((df['High'] > df['High'].shift(1)) & (df['High'] - df['High'].shift(1) > df['Low'].shift(1) - df['Low']), df['High'] - df['High'].shift(1), 0)
    df['-dm'] = np.where((df['Low'] < df['Low'].shift(1)) & (df['High'].shift(1) - df['High'] < df['Low'].shift(1) - df['Low']), df['Low'].shift(1) - df['Low'], 0)
    df['tr_ema'] = df['tr'].ewm(span=n, adjust=False).mean()
    df['+dm_ema'] = df['+dm'].ewm(span=n, adjust=False).mean()
    df['-dm_ema'] = df['-dm'].ewm(span=n, adjust=False).mean()
    df['+di'] = (df['+dm_ema'] / df['tr_ema']) * 100
    df['-di'] = (df['-dm_ema'] / df['tr_ema']) * 100
    df['dx'] = (abs(df['+di'] - df['-di']) / (df['+di'] + df['-di'])) * 100
    df['adx'] = df['dx'].rolling(window=n).mean()

    return df

# calculates MACD
def macd_calc(df, short_n, long_n, signal_n):
    df['ema_short'] = df['Close'].ewm(span=short_n, adjust=False).mean()
    df['ema_long'] = df['Close'].ewm(span=long_n, adjust=False).mean()
    df['macd_line'] = df['ema_short'] - df['ema_long']
    df['signal_line'] = df['macd_line'].ewm(span=signal_n, adjust=False).mean()
    df['macd_histogram'] = df['macd_line'] - df['signal_line']

    return df

def find_extrema(df, n):
    df['min'] = df.iloc[argrelextrema(df['Close'].values, np.less_equal, order=n)[0]]['Close']
    df['max'] = df.iloc[argrelextrema(df['Close'].values, np.greater_equal, order=n)[0]]['Close']
    return df


In [ ]:
df=find_extrema(df,16)
df=ma_calc(df,524)
df=longshort_ma_calc(df,564,4)
df=ema_calc(df,63)
df=longshort_ema_calc(df,374,5)
df=rsi_calc(df,118)
df=adx_calc(df,32)

# columns_to_drop = ['hl_diff', 'hc_diff', 'lc_diff', 'tr', '+dm', '-dm', 'tr_ema', '+dm_ema', '-dm_ema', '+di', '-di', 'dx']

# # Drop the columns
# df = df.drop(columns=columns_to_drop)


In [ ]:
 # Initialize the signal column with hold signals
df['true_signal'] = 0

# Generate buy signals at local minima
df.loc[df['min'].notna(), 'true_signal'] = 1

# Generate sell signals at local maxima
df.loc[df['max'].notna(), 'true_signal'] = -1


In [ ]:
df.true_signal.value_counts()

 0    2390
 1      55
-1      52
Name: true_signal, dtype: int64

In [ ]:
# As the buy/sell signals themselves are sparse,I'd predict positions instead of signals
for day in np.unique(df.index.date):
    indices = df[df.index.date == day].index
    for i in range(len(indices) - 1):  # subtracting 1 because we're looking ahead by 1 row
        if df.loc[indices[i], "true_signal"] == 1 and df.loc[indices[i + 1], "true_signal"] == 0:
            df.loc[indices[i + 1], "true_signal"] = 1
        elif df.loc[indices[i], "true_signal"] == -1 and df.loc[indices[i + 1], "true_signal"] == 0:
            df.loc[indices[i + 1], "true_signal"] = -1


In [ ]:
df.true_signal.value_counts()

 0    971
-1    811
 1    715
Name: true_signal, dtype: int64

In [ ]:
df["longshort_sma_delta"]=df["sma_long"]-df["sma_short"]
df["longshort_ema_delta"]=df["ema_long"]-df["ema_short"]
df["Close_sma_delta"]=df["Close"]-df["sma_short"]
df["Close_ema_delta"]=df["Close"]-df["ema_short"]

In [ ]:
# 1) Body Size
df['body_size'] = abs(df['Close'] - df['Open'])

# 2) Upper Shadow
df['upper_shadow'] = df['High'] - df[['Open', 'Close']].max(axis=1)

# 3) Lower Shadow
df['lower_shadow'] = df[['Open', 'Close']].min(axis=1) - df['Low']

# 4) Candle Direction
df['candle_direction'] = np.where(df['Close'] >= df['Open'], 1, -1)

# 5) Price Range
df['price_range'] = df['High'] - df['Low']

# 6) Relative Price / Normalized Price
df['relative_price'] = (df['Close'] - df['Low']) / (df['High'] - df['Low'])


In [ ]:
df['VWAP'] = (df['Volume'] * (df['High'] + df['Low'] + df['Close']) / 3).cumsum() / df['Volume'].cumsum()
df['OBV'] = (df['Volume'] * (~df['Close'].diff().le(0) * 2 - 1)).cumsum()
df['PVT'] = (df['Volume'] * (df['Close'].pct_change())).cumsum()
typical_price = (df['High'] + df['Low'] + df['Close']) / 3
money_flow = typical_price * df['Volume']
df['MFI'] = money_flow.rolling(window=window_size).sum() / df['Volume'].rolling(window=window_size).sum()


In [ ]:
df=df.drop(["Close","High","Low"],axis=1)

In [ ]:
# df=df.fillna(-1)

In [ ]:
train=df[:38*55]
test=df[38*56:]

In [ ]:
len(test)

369

In [ ]:
target=train.true_signal
indicators=train.drop(["true_signal"],axis=1)

In [ ]:
val_target=test.true_signal
val_indicators=test.drop(["true_signal"],axis=1)

In [ ]:
target.value_counts()

 0    869
-1    651
 1    570
Name: true_signal, dtype: int64

In [ ]:
# !pip -q install flaml
from flaml import AutoML
automl = AutoML()
automl.fit(indicators, target, X_val=val_indicators,y_val=val_target,task="classification", max_iter=1000,metric='micro_f1',estimator_list=["lgbm"])


[flaml.automl.logger: 03-18 08:26:03] {1680} INFO - task = classification
[flaml.automl.logger: 03-18 08:26:03] {1688} INFO - Data split method: stratified
[flaml.automl.logger: 03-18 08:26:03] {1691} INFO - Evaluation method: holdout
[flaml.automl.logger: 03-18 08:26:03] {1789} INFO - Minimizing error metric: 1-micro_f1
[flaml.automl.logger: 03-18 08:26:03] {1901} INFO - List of ML learners in AutoML Run: ['lgbm']
[flaml.automl.logger: 03-18 08:26:03] {2219} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 03-18 08:26:04] {2345} INFO - Estimated sufficient time budget=10000s. Estimated necessary time budget=10s.
[flaml.automl.logger: 03-18 08:26:04] {2392} INFO -  at 0.8s,	estimator lgbm's best error=0.6152,	best estimator lgbm's best error=0.6152
[flaml.automl.logger: 03-18 08:26:04] {2219} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 03-18 08:26:04] {2392} INFO -  at 1.0s,	estimator lgbm's best error=0.6152,	best estimator lgbm's best error=0.6152
[fl

In [ ]:
df=df.fillna(-1)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Precision, Recall
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Assuming df is your DataFrame and 'true_signal' is the target column
target = 'true_signal'
features = df.drop(target, axis=1)
labels = df[target]

# Apply MinMaxScaler to the features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)

# One-hot encode labels
y = to_categorical(labels+1)  # Adding 1 to shift the labels to 0, 1, 2

# Split the data into train and test sets
train_size = int(len(scaled_features) * 0.7)
test_size = len(scaled_features) - train_size
X_train, X_test = scaled_features[0:train_size,:], scaled_features[train_size:len(scaled_features),:]
y_train, y_test = y[0:train_size], y[train_size:len(y)]

# Define the model
model = Sequential()
model.add(Dense(680, input_shape=(X_train.shape[1],)))  # Freeze this layer
model.add(Dropout(0.5))  # Add dropout for regularization
model.add(BatchNormalization())  # Add batch normalization
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))  # Add dropout for regularization
model.add(BatchNormalization())  # Add batch normalization
model.add(Dense(3, activation='softmax'))  # Assuming there are 3 classes (0, 1, 2)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])

# Calculate class weights
total_samples = len(labels)
class_samples = np.bincount(labels+1)
num_classes = len(class_samples)
class_weights = {i: total_samples / (num_classes * class_samples[i]) for i in range(num_classes)}

# Train the model with validation data
model.fit(X_train, y_train, epochs=1000, validation_data=(X_test, y_test), class_weight=class_weights)

# Make predictions
yhat = model.predict(X_test, verbose=0)


Epoch 1/1000
55/55 [==============================] - 3s 13ms/step - loss: 1.2818 - accuracy: 0.4642 - precision: 0.4723 - recall: 0.4001 - val_loss: 1.0334 - val_accuracy: 0.4280 - val_precision: 0.5463 - val_recall: 0.1573
Epoch 2/1000
55/55 [==============================] - 0s 6ms/step - loss: 1.1606 - accuracy: 0.4865 - precision: 0.5066 - recall: 0.4156 - val_loss: 1.0488 - val_accuracy: 0.4680 - val_precision: 0.5239 - val_recall: 0.2627
Epoch 3/1000
55/55 [==============================] - 1s 9ms/step - loss: 1.0528 - accuracy: 0.5192 - precision: 0.5577 - recall: 0.4511 - val_loss: 1.0677 - val_accuracy: 0.4853 - val_precision: 0.5553 - val_recall: 0.3013
Epoch 4/1000
55/55 [==============================] - 1s 11ms/step - loss: 0.9995 - accuracy: 0.5478 - precision: 0.5743 - recall: 0.4602 - val_loss: 1.0765 - val_accuracy: 0.4813 - val_precision: 0.5599 - val_recall: 0.3987
Epoch 5/1000
55/55 [==============================] - 1s 11ms/step - loss: 0.9599 - accuracy: 0.5484 -

This didn't work again, going back to something as simple as using open, high, low and close with 5 indicators with standard windows  and then using LSTMs

In [7]:
import numpy as np
import pandas as pd
from scipy import signal
from scipy.stats import skew, kurtosis
import pywt
from scipy.signal import argrelextrema

In [8]:
df=pd.read_csv("/content/Manappuram_10minute.csv")
df.tail(5)
# Converting Date column into datetime dftype
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)


In [9]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

In [10]:
# calculates SMA using "n" as the rolling window size
def ma_calc(df, n):
  df["sma"] = df.Close.rolling(window=n).mean()
  return df

# calculates SMA using "m" and "n" as the rolling window size
def longshort_ma_calc(df, m, n):
  df["sma_short"] = df.Close.rolling(window=min(m,n)).mean()
  df["sma_long"] = df.Close.rolling(window=max(m,n)).mean()
  return df

# calculates EMA using "n" as the rolling window size
def ema_calc(df, n):
  df["ema"] = df.Close.ewm(span=n, adjust=False).mean()
  return df


def longshort_ema_calc(df, m, n):
  df["ema_short"] = df.Close.ewm(span=min(m,n), adjust=False).mean()
  df["ema_long"] = df.Close.ewm(span=max(m,n), adjust=False).mean()
  return df

# calculates RSI using "n" as the lookback period
def rsi_calc(df, n):
  df['rsi'] = 100 - (100 / (1 + df['Close'].diff().apply(lambda x: x if x > 0 else 0).rolling(window=n).mean() / df['Close'].diff().apply(lambda x: -x if x < 0 else 0).rolling(window=n).mean()))
  return df

# calculates OBV
def obv_calc(df):
  df['obv'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
  return df

def bb_calc(df, n):
  df["sma"] = df.Close.rolling(window=n).mean()
  df["std"] = df.Close.rolling(window=n).std()
  df["upper_bb"] = df["sma"] + (2 * df["std"])
  df["lower_bb"] = df["sma"] - (2 * df["std"])
  return df

# Volume weighted average price
def vwap_calc(df):
    df['vwap'] = (df['Volume'] * df['Close']).cumsum() / df['Volume'].cumsum()
    return df

# Supertrend Indicator
def supertrend_calc(df, period, multiplier):
    # Calculate basic upper and lower bands
    df['hl_avg'] = (df['High'] + df['Low']) / 2
    df['range'] = df['High'] - df['Low']
    df['upper_band'] = df['hl_avg'] + multiplier * df['range']
    df['lower_band'] = df['hl_avg'] - multiplier * df['range']

    # Calculate final upper and lower bands
    df['upper_band_final'] = np.where((df['upper_band'] < df['upper_band'].shift(1)) | (df['Close'] > df['upper_band'].shift(1)), df['upper_band'], df['upper_band'].shift(1))
    df['lower_band_final'] = np.where((df['lower_band'] > df['lower_band'].shift(1)) | (df['Close'] < df['lower_band'].shift(1)), df['lower_band'], df['lower_band'].shift(1))

    # Calculate Supertrend
    df['supertrend'] = np.where(df['Close'] <= df['upper_band_final'], df['upper_band_final'], df['lower_band_final'])
    df['supertrend'] = np.where(df['Close'] >= df['lower_band_final'], df['lower_band_final'], df['supertrend'])

    return df

# calculates Average Directional Index (ADX)
def adx_calc(df, n):
    df['hl_diff'] = df['High'] - df['Low']
    df['hc_diff'] = abs(df['High'] - df['Close'].shift(1))
    df['lc_diff'] = abs(df['Low'] - df['Close'].shift(1))
    df['tr'] = df[['hl_diff', 'hc_diff', 'lc_diff']].max(axis=1)
    df['+dm'] = np.where((df['High'] > df['High'].shift(1)) & (df['High'] - df['High'].shift(1) > df['Low'].shift(1) - df['Low']), df['High'] - df['High'].shift(1), 0)
    df['-dm'] = np.where((df['Low'] < df['Low'].shift(1)) & (df['High'].shift(1) - df['High'] < df['Low'].shift(1) - df['Low']), df['Low'].shift(1) - df['Low'], 0)
    df['tr_ema'] = df['tr'].ewm(span=n, adjust=False).mean()
    df['+dm_ema'] = df['+dm'].ewm(span=n, adjust=False).mean()
    df['-dm_ema'] = df['-dm'].ewm(span=n, adjust=False).mean()
    df['+di'] = (df['+dm_ema'] / df['tr_ema']) * 100
    df['-di'] = (df['-dm_ema'] / df['tr_ema']) * 100
    df['dx'] = (abs(df['+di'] - df['-di']) / (df['+di'] + df['-di'])) * 100
    df['adx'] = df['dx'].rolling(window=n).mean()

    return df

# calculates MACD
def macd_calc(df, short_n, long_n, signal_n):
    df['ema_short'] = df['Close'].ewm(span=short_n, adjust=False).mean()
    df['ema_long'] = df['Close'].ewm(span=long_n, adjust=False).mean()
    df['macd_line'] = df['ema_short'] - df['ema_long']
    df['signal_line'] = df['macd_line'].ewm(span=signal_n, adjust=False).mean()
    df['macd_histogram'] = df['macd_line'] - df['signal_line']

    return df

def find_extrema(df, n):
    df['min'] = df.iloc[argrelextrema(df['Close'].values, np.less_equal, order=n)[0]]['Close']
    df['max'] = df.iloc[argrelextrema(df['Close'].values, np.greater_equal, order=n)[0]]['Close']
    return df


In [11]:
df=find_extrema(df,16)
df=ma_calc(df,524)
df=longshort_ma_calc(df,564,4)
df=ema_calc(df,63)
df=longshort_ema_calc(df,374,5)
df=rsi_calc(df,118)
df=adx_calc(df,32)

# columns_to_drop = ['hl_diff', 'hc_diff', 'lc_diff', 'tr', '+dm', '-dm', 'tr_ema', '+dm_ema', '-dm_ema', '+di', '-di', 'dx']

# # Drop the columns
# df = df.drop(columns=columns_to_drop)


In [12]:
 # Initialize the signal column with hold signals
df['true_signal'] = 0

# Generate buy signals at local minima
df.loc[df['min'].notna(), 'true_signal'] = 1

# Generate sell signals at local maxima
df.loc[df['max'].notna(), 'true_signal'] = -1


In [13]:
# As the buy/sell signals themselves are sparse,I'd predict positions instead of signals
for day in np.unique(df.index.date):
    indices = df[df.index.date == day].index
    for i in range(len(indices) - 1):  # subtracting 1 because we're looking ahead by 1 row
        if df.loc[indices[i], "true_signal"] == 1 and df.loc[indices[i + 1], "true_signal"] == 0:
            df.loc[indices[i + 1], "true_signal"] = 1
        elif df.loc[indices[i], "true_signal"] == -1 and df.loc[indices[i + 1], "true_signal"] == 0:
            df.loc[indices[i + 1], "true_signal"] = -1


In [14]:
df.true_signal.value_counts()

 0    971
-1    811
 1    715
Name: true_signal, dtype: int64

In [15]:
df

,Open,High,Low,Close,Volume,min,max,sma,sma_short,sma_long,...,+dm,-dm,tr_ema,+dm_ema,-dm_ema,+di,-di,dx,adx,true_signal
Date,,,,,,,,,,,,,,,,,,,,,
2020-01-21 09:15:00,179.30,180.20,178.25,180.15,173897,NaN,NaN,NaN,NaN,NaN,...,0.00,0.0,1.950000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0
2020-01-21 09:25:00,180.00,181.30,180.00,180.50,175277,NaN,NaN,NaN,NaN,NaN,...,1.10,0.0,1.910606,0.066667,0.000000,3.489294,0.000000,100.000000,NaN,0
2020-01-21 09:35:00,180.50,181.05,180.25,180.55,110920,NaN,NaN,NaN,NaN,NaN,...,0.00,0.0,1.843297,0.062626,0.000000,3.397514,0.000000,100.000000,NaN,0
2020-01-21 09:45:00,180.55,181.50,180.05,181.35,80456,NaN,181.35,NaN,180.6375,NaN,...,0.45,0.2,1.819460,0.086103,0.012121,4.732362,0.666198,75.319414,NaN,-1
2020-01-21 09:55:00,181.35,181.65,181.00,181.30,73996,NaN,NaN,NaN,180.9250,NaN,...,0.15,0.0,1.748584,0.089976,0.011387,5.145648,0.651189,77.532943,NaN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-28 14:35:00,123.90,124.25,123.70,124.15,289726,NaN,NaN,107.540935,123.8500,106.497872,...,0.00,0.0,0.773377,0.248342,0.026264,32.111333,3.396051,80.871298,79.639203,0
2020-04-28 14:45:00,124.15,125.80,124.10,125.80,1706683,NaN,NaN,107.598187,124.3500,106.556915,...,1.55,0.0,0.829536,0.327230,0.024673,39.447359,2.974254,85.977650,80.222201,0
2020-04-28 14:55:00,125.80,125.80,125.80,125.80,145240,NaN,NaN,107.656489,124.8875,106.616046,...,0.00,0.0,0.779261,0.307398,0.023177,39.447359,2.974254,85.977650,80.587223,0


In [16]:
df=df.fillna(-100)
target=df["true_signal"]
data_set=df.drop("true_signal",axis=1)

In [17]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
data_set_scaled = sc.fit_transform(data_set)
print(data_set_scaled)

[[0.87199313 0.8744045  0.8869936  ... 0.         0.         0.        ]
 [0.87800687 0.88393244 0.90191898 ... 0.         1.         0.        ]
 [0.88230241 0.881767   0.90405117 ... 0.         1.         0.        ]
 ...
 [0.41237113 0.40320485 0.43965885 ... 0.05218798 0.92988825 0.99388848]
 [0.41237113 0.45257687 0.43965885 ... 0.03545593 0.96680991 0.99694424]
 [0.45661512 0.44781291 0.47206823 ... 0.03280426 0.96680991 1.        ]]


In [18]:
# Set the number of backcandles
backcandles = 38

# Initialize an empty list for X
X = []

# Loop over the first 7 columns of data_set_scaled
for j in range(len(data_set.columns)):
    X.append([data_set_scaled[i-backcandles:i, j] for i in range(backcandles, data_set_scaled.shape[0])])

# Convert X to a numpy array and move the first axis to the third position
X = np.moveaxis(np.array(X), 0, 2)


# Create the target array y
yi = np.array(target)
y = np.reshape(yi[backcandles:], (len(yi) - backcandles, 1))  # Exclude the first 'backcandles' entries


# Print the shapes of X and y
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

# split data into train test sets
splitlimit = int(len(X)*0.8)
print(splitlimit)
X_train, X_test = X[:splitlimit], X[splitlimit:]
y_train, y_test = y[:splitlimit], y[splitlimit:]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_train)

Shape of X: (2459, 38, 27)
Shape of y: (2459, 1)
1967
(1967, 38, 27)
(492, 38, 27)
(1967, 1)
(492, 1)
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [1]]


In [19]:
np.unique(y_train)

array([-1,  0,  1])

In [20]:
X_train.shape,X_test.shape

((1967, 38, 27), (492, 38, 27))

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense,Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.keras import metrics
from sklearn.metrics import classification_report
from keras.regularizers import l2



# Assuming y_train is your target variable
y_train_one_hot = to_categorical(y_train, num_classes=3)
y_test_one_hot = to_categorical(y_test, num_classes=3)

# Create a Sequential model
model = Sequential()
model.add(LSTM(512, input_shape=(backcandles, 27),kernel_regularizer=l2(0.2),activity_regularizer=l2(0.2)))
model.add(Dropout(0.5))
model.add(Dense(3, activation="softmax"))


model.compile(optimizer=Adam(), loss='categorical_crossentropy',metrics=['accuracy', metrics.Precision(), metrics.Recall()])

early_stopping = EarlyStopping(monitor='val_loss', patience=1000, verbose=1)

model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)


model.fit(x=X_train, y=y_train_one_hot, batch_size=15, epochs=10000, shuffle=True, validation_data=(X_test, y_test_one_hot), callbacks=[early_stopping, model_checkpoint])

# Loading the best weights
model.load_weights('best_model.h5')

# Making predictions on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Print precision, recall, and F1-score for each class
print(classification_report(np.argmax(y_test_one_hot, axis=1), y_pred_classes))

Epoch 1/10000
132/132 [==============================] - ETA: 0s - loss: 2.4524 - accuracy: 0.4545 - precision_1: 0.5255 - recall_1: 0.0366
Epoch 1: val_accuracy improved from -inf to 0.21545, saving model to best_model.h5
132/132 [==============================] - 5s 14ms/step - loss: 2.4524 - accuracy: 0.4545 - precision_1: 0.5255 - recall_1: 0.0366 - val_loss: 1.1733 - val_accuracy: 0.2154 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/10000
 17/132 [==>...........................] - ETA: 0s - loss: 1.1279 - accuracy: 0.3961 - precision_1: 1.0000 - recall_1: 0.0039        

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


127/132 [===========================>..] - ETA: 0s - loss: 1.0938 - accuracy: 0.4525 - precision_1: 0.5895 - recall_1: 0.0588
Epoch 2: val_accuracy did not improve from 0.21545
132/132 [==============================] - 1s 8ms/step - loss: 1.0941 - accuracy: 0.4540 - precision_1: 0.5895 - recall_1: 0.0569 - val_loss: 1.1525 - val_accuracy: 0.2154 - val_precision_1: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/10000
127/132 [===========================>..] - ETA: 0s - loss: 1.0882 - accuracy: 0.4404 - precision_1: 0.5880 - recall_1: 0.0667
Epoch 3: val_accuracy did not improve from 0.21545
132/132 [==============================] - 1s 8ms/step - loss: 1.0884 - accuracy: 0.4392 - precision_1: 0.5880 - recall_1: 0.0696 - val_loss: 1.2684 - val_accuracy: 0.2154 - val_precision_1: 0.2303 - val_recall_1: 0.1606
Epoch 4/10000
128/132 [============================>.] - ETA: 0s - loss: 1.0854 - accuracy: 0.4490 - precision_1: 0.5968 - recall_1: 0.0979
Epoch 4: val_accuracy did not improve from